# Using Parliament's APIs

On [data.parliament.uk](http://www.data.parliament.uk) Parliament has released a large number of APIs to aid access to their data. A full list of APIs can be found at [explore.data.parliament.uk](http://explore.data.parliament.uk). Note: most of these APIs are currently beta releases and may change.

On Parliamentary questions the are a number of APIs:
- Written questions to the House of Commons [(documentation here)](http://explore.data.parliament.uk/?learnmore=Commons%20Written%20Questions)and to the House of Lords [(documentation here)](http://explore.data.parliament.uk/?learnmore=Lords%20Written%20Questions).
- Answered written questions to both Houses [(documentation here)](http://explore.data.parliament.uk/?learnmore=Parliamentary%20Questions%20Answered) and to individual Houses [(e.g. the House of Commons here)](http://explore.data.parliament.uk/?learnmore=Commons%20Answered%20Questions).
- Oral questions to the Commons [(documentation here)](http://explore.data.parliament.uk/?learnmore=Commons%20Oral%20Questions) and to the Lords [(documentation here)](http://explore.data.parliament.uk/?learnmore=Lords%20Oral%20Questions).

In this notebook I will briefly demonstate how to use the Commons Written Questions API and the Parliamentary Questions Answered API using python. The other Parliamentary questions APIs follow a similiar pattern to these two so it should be straightforward to transfer these examples. 

In [1]:
import requests
import json

from math import ceil
from __future__ import division

## Written questions to the House of Commons

Following the document [(documentation here)](http://explore.data.parliament.uk/?learnmore=Commons%20Written%20Questions) there are a number of different ways to query the data. First let's select all written questions sent to a given department.

### Selecting a given department
We can query the questions sent to a given deparment by using the following code where the results are saved to a dictionary for ease of use.

In [2]:
commons_url = "http://lda.data.parliament.uk/commonswrittenquestions.json"

params = {}
params["AnsweringBody"]="HM Treasury"


response = requests.get(commons_url, params=params).json()

The number of hits matching our search criteria can be printed, as can the questions themselves.

In [3]:
print "Number of hits:", response["result"]["totalResults"]

Number of hits: 11161


### Selecting a given department and filter by date

As well as selecting a given department we may want to limit date range considered. Below we select HM Treasury and only bring back questions asked in 2017.

In [4]:
commons_url = "http://lda.data.parliament.uk/commonswrittenquestions/tabled.json"

params = {}
params["AnsweringBody"] = "HM Treasury"
params["startDate"] = "2017-01-01"
params["endDate"] = "2018-01-01"

response = requests.get(commons_url, params=params).json()

In [5]:
total_results = response["result"]["totalResults"]
print "Number of hits:", total_results

Number of hits: 2561


As expecting their are fewer results. Let's have a look at the first 100 characters of the questions asked.

In [6]:
for indx, line in enumerate(response["result"]["items"]):
    print indx, line["questionText"][0:100]

0 To ask Mr Chancellor of the Exchequer, if it remains his policy to implement the Second Payment Serv
1 To ask Mr Chancellor of the Exchequer, what assessment he has made of the potential merits of extend
2 To ask Mr Chancellor of the Exchequer, what the title was of each set of regulations introduced by h
3 To ask Mr Chancellor of the Exchequer, what processes his Department has put in place to (a) monitor
4 To ask Mr Chancellor of the Exchequer, what information his Department holds on the number of employ
5 To ask Mr Chancellor of the Exchequer, what assessment he has made of the effect of the proposed clo
6 To ask Mr Chancellor of the Exchequer, what assessment he has made of the effect of the proposed Roy
7 To ask Mr Chancellor of the Exchequer, Energy and Industrial Strategy, when his Department was first
8 To ask Mr Chancellor of the Exchequer, which EU (a) Directives, (b) Regulations and (c) other legisl
9 To ask Mr Chancellor of the Exchequer, what his Department's estimate i

So whilst there are thousands of hits, only ten results have been returned. This is because the API returns the results in a series of pages, each page only containering a given number of records.

### Extracting all the results

In practice we will want to extract all the hits. To do so, first only return one hit so that we can determine the size of the data.

In [7]:
params = {}
params["AnsweringBody"] = "HM Treasury"
params["startDate"] = "2017-01-01"
params["endDate"] = "2018-01-01"
params["_pageSize"] = 1

response = requests.get(commons_url, params=params).json()
total_results = response["result"]["totalResults"]

Next consider how many records it is sensible to download at any one time, this will be the page size. The API limits this to a maximum of 500.

In [8]:
page_size = 200
params["_pageSize"] = page_size
num_pages = int(ceil(total_results / page_size))

print "Number of pages to download:", num_pages

Number of pages to download: 13


Then loop over the number of pages present, saving the questions.

In [9]:
questions = []

for page in xrange(0, num_pages):            
        params["_page"] = page
        results = requests.get(commons_url, params=params).json()
        for line in results["result"]["items"]:
            questions.append(line["questionText"])

How many questions have we downloaded?

In [10]:
len(questions)

2561

## Answered Parliamentary questions

This API is a little different and contains more fields for each record. The code below shows how to write a query similiar to the previous one but using this different API.

In [11]:
commons_url = "http://lda.data.parliament.uk/answeredquestions.json"

params = {}
params["AnsweringBody"] = "HM Treasury"
params["minEx-date"] = "2017-01-01"
params["maxEx-date"] = "2018-01-01"

response = requests.get(commons_url, params=params).json()
response["result"]["totalResults"]

2907

This is considerding answered questions from both houses so it is not surprising that that the numbers differ.